# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [51]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [52]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.42,60,97,6.23,TF,1714881665
1,1,longyearbyen,78.2186,15.6401,-4.09,74,75,1.54,SJ,1714881665
2,2,kodiak,57.7900,-152.4072,5.25,80,98,3.35,US,1714881665
3,3,waitangi,-43.9535,-176.5597,12.12,86,83,3.13,NZ,1714881666
4,4,port mathurin,-19.6833,63.4167,25.81,72,16,8.60,MU,1714881666


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [75]:
%%capture --no-display

# Configure the map plot
humidity_plot_by_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoLight",
    frame_width = 750,
    frame_height = 500,
    size = "Humidity",   
    color = "City"
)

# Display the map
humidity_plot_by_city


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [54]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal conditions for traveling 
    #A max temperature lower than 27 degrees but higher than 21
    #Wind speed less than 4.5 m/s
    #Zero cloudiness
#-----------------------------------------
city_data_df_filtered=city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0),:]

# Drop any rows with null values
city_data_df_filtered=city_data_df_filtered.dropna()                                                          

# Display sample data
city_data_df_filtered


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,ghat,24.9647,10.1728,25.26,33,0,2.64,LY,1714881683
160,160,tindouf,27.6711,-8.1474,21.25,23,0,3.60,DZ,1714881484
179,179,ciudad lazaro cardenas,17.9583,-102.2000,23.88,83,0,1.19,MX,1714881697
200,200,lamar,33.6668,-95.5836,23.13,88,0,3.09,US,1714881701
257,257,bacalar,18.6667,-88.3833,25.04,87,0,2.49,MX,1714881711
265,265,ilhabela,-23.7781,-45.3581,24.26,66,0,1.66,BR,1714881523
349,349,ormond-by-the-sea,29.3491,-81.0665,23.94,83,0,0.00,US,1714881727
434,434,lazaro cardenas,17.9583,-102.2000,23.88,83,0,1.19,MX,1714881697
441,441,namibe,-15.1961,12.1522,21.99,72,0,2.60,AO,1714881745
498,498,umluj,25.0213,37.2685,25.47,49,0,2.85,SA,1714881647


### Step 3: Create a new DataFrame called `hotel_df`.

In [55]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filtered[["City","Country","Lat","Lng","Humidity"]].copy().reset_index(drop=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head(100)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ghat,LY,24.9647,10.1728,33,
1,tindouf,DZ,27.6711,-8.1474,23,
2,ciudad lazaro cardenas,MX,17.9583,-102.2000,83,
3,lamar,US,33.6668,-95.5836,88,
4,bacalar,MX,18.6667,-88.3833,87,
5,ilhabela,BR,-23.7781,-45.3581,66,
6,ormond-by-the-sea,US,29.3491,-81.0665,83,
7,lazaro cardenas,MX,17.9583,-102.2000,83,
8,namibe,AO,-15.1961,12.1522,72,
9,umluj,SA,25.0213,37.2685,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000
limit=20
categories=f"accommodation.hotel"

params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude= hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghat - nearest hotel: فندق تاسيلي
tindouf - nearest hotel: محمد بوسبي
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
lamar - nearest hotel: 1st St Lofts
bacalar - nearest hotel: El Paraíso
ilhabela - nearest hotel: Vila Kebaya
ormond-by-the-sea - nearest hotel: Howard Johnson
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
namibe - nearest hotel: Nelsal Pensao
umluj - nearest hotel: No hotel found
yanchep - nearest hotel: Yanchep Inn
ad dabbah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,ghat,LY,24.9647,10.1728,33,فندق تاسيلي
1,tindouf,DZ,27.6711,-8.1474,23,محمد بوسبي
2,ciudad lazaro cardenas,MX,17.9583,-102.2000,83,Hotel Sol del Pacífico
3,lamar,US,33.6668,-95.5836,88,1st St Lofts
4,bacalar,MX,18.6667,-88.3833,87,El Paraíso
5,ilhabela,BR,-23.7781,-45.3581,66,Vila Kebaya
6,ormond-by-the-sea,US,29.3491,-81.0665,83,Howard Johnson
7,lazaro cardenas,MX,17.9583,-102.2000,83,Hotel Sol del Pacífico
8,namibe,AO,-15.1961,12.1522,72,Nelsal Pensao
9,umluj,SA,25.0213,37.2685,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoLight",
    frame_width = 750,
    frame_height = 500,
    color = "City",
    hover_cols=["Lng","Lat","Humidity","Country","Hotel Name"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)